In [117]:
#Import Pandas to parse the dataframe
import pandas as pd

#Used to download the sampledata module
#import bokeh as bo
#bo.sampledata.download()

#Import the necessary Bokeh tools
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import WheelPanTool, WheelZoomTool, Range1d, Scatter, ColumnDataSource
import bokeh.sampledata.stocks as stocks

#Get the data
df = pd.DataFrame(stocks.MSFT)[60:120]
df["date"] = pd.to_datetime(df["date"])

dates = df.date.to_list()

def foreach(fn,iterable):
    for x in iterable:
        fn(x)

#Set up the candlestick graph
inc = df.close > df.open
dec = df.open > df.close
w = 16*60*60*1000 # milliseconds

#Define the tools on the graph (Has been replaced)
#TOOLS = "xwheel_zoom"

#Create a function to make the figures
def makeplot(is_line, is_candlestick, range):
    #Create the figure
    p = figure(x_axis_type="datetime", width=1000, height=150,
           title="Graph", background_fill_color="#efefef", tools="", toolbar_location=None)
    
    p.sizing_mode="inherit"
    p.xaxis.major_label_orientation = 0.8 # radians


    #Configure its properties
    if range != 0: 
        p.x_range=range
    else:
        zoom = WheelZoomTool(dimensions="width")
        p.add_tools(zoom)
        pan = WheelPanTool(dimension="width")
        p.add_tools(pan)
        p.toolbar.active_scroll = zoom
        p.x_range = Range1d(dates[0], dates[-1], bounds=(dates[0], dates[-1]))
        p.toolbar_location = "above"
    
    p.title = f'MSFT {"candlestick" if is_candlestick else ""}{" + " if is_candlestick and is_line else ""}{"line" if is_line else ""}'

    #Add the candlestick graph
    if is_candlestick:
        p.segment(df.date, df.high, df.date, df.low, color="black")

        p.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#eb3c40")
        p.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="white",
        line_color="#49a3a3", line_width=2)

        colors = []
        foreach((lambda x : colors.append("#eb3c40" if x else "white")), dec)
        scatter = Scatter(x="x", y="y", size=5, fill_color=colors,marker="square")
        source = ColumnDataSource({"x": df.date, "y": df.high})
        p.add_glyph(source, scatter)

    #Add the line graph
    if is_line:
        p.line(df.date, df.high, line_width=2)
    return p

#Make the figures
candlestick_line = makeplot(True, True, 0)
candlestick = makeplot(False, True, candlestick_line.x_range)
line = makeplot(True, False, candlestick_line.x_range)

#Display the figures
display_column=column(candlestick_line, candlestick, line)
display_column.sizing_mode="stretch_both"
show(display_column)

ValueError: failed to validate Scatter(id='p14257', ...).fill_color: expected an element of either String, Nullable(Color), Instance(Value), Instance(Field), Instance(Expr), Struct(value=Nullable(Color), transform=Instance(Transform)), Struct(field=String, transform=Instance(Transform)) or Struct(expr=Instance(Expression), transform=Instance(Transform)), got ['#eb3c40', '#eb3c40', 'white', '#eb3c40', 'white', 'white', 'white', 'white', 'white', '#eb3c40', '#eb3c40', '#eb3c40', 'white', 'white', 'white', '#eb3c40', 'white', 'white', 'white', '#eb3c40', '#eb3c40', 'white', '#eb3c40', '#eb3c40', '#eb3c40', 'white', 'white', '#eb3c40', 'white', 'white', '#eb3c40', 'white', 'white', 'white', '#eb3c40', '#eb3c40', 'white', '#eb3c40', 'white', '#eb3c40', '#eb3c40', '#eb3c40', '#eb3c40', 'white', '#eb3c40', '#eb3c40', '#eb3c40', 'white', 'white', '#eb3c40', '#eb3c40', 'white', 'white', '#eb3c40', 'white', '#eb3c40', '#eb3c40', '#eb3c40', 'white', '#eb3c40']